# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 11:13AM,258042,10,9196066,"Citieffe, Inc.",Released
1,Feb 23 2023 11:01AM,258041,10,9196061,ISDIN Corporation,Released
2,Feb 23 2023 11:00AM,258039,10,8876506-BO,Eywa Pharma Inc.,Released
3,Feb 23 2023 11:00AM,258039,10,9191748,Eywa Pharma Inc.,Released
4,Feb 23 2023 11:00AM,258039,10,9191785,Eywa Pharma Inc.,Released
5,Feb 23 2023 11:00AM,258039,10,9191786,Eywa Pharma Inc.,Released
6,Feb 23 2023 11:00AM,258039,10,9191787,Eywa Pharma Inc.,Released
7,Feb 23 2023 11:00AM,258039,10,9191788,Eywa Pharma Inc.,Released
8,Feb 23 2023 11:00AM,258039,10,7627605,Eywa Pharma Inc.,Released
9,Feb 23 2023 11:00AM,258039,10,9193479,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,258037,Released,5
33,258038,Released,1
34,258039,Released,20
35,258041,Released,1
36,258042,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258037,NaN,NaN,5.0
258038,NaN,NaN,1.0
258039,NaN,NaN,20.0
258041,NaN,NaN,1.0
258042,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0.0,0.0,13.0
257936,0.0,11.0,12.0
257942,0.0,18.0,7.0
257943,0.0,0.0,1.0
257981,3.0,5.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0,0,13
257936,0,11,12
257942,0,18,7
257943,0,0,1
257981,3,5,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,0,0,13
1,257936,0,11,12
2,257942,0,18,7
3,257943,0,0,1
4,257981,3,5,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,,,13
1,257936,,11,12
2,257942,,18,7
3,257943,,,1
4,257981,3,5,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 11:13AM,258042,10,"Citieffe, Inc."
1,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation
2,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.
22,Feb 23 2023 10:55AM,258038,15,Lonza
23,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC
28,Feb 23 2023 10:21AM,258035,10,MedStone Pharma LLC
29,Feb 23 2023 9:52AM,258031,10,ISDIN Corporation
36,Feb 23 2023 9:43AM,258030,10,"Methapharm, Inc."
39,Feb 23 2023 9:41AM,258029,20,Granules Consumer Health
40,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 11:13AM,258042,10,"Citieffe, Inc.",,,1
1,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation,,,1
2,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.,,,20
3,Feb 23 2023 10:55AM,258038,15,Lonza,,,1
4,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC,,,5
5,Feb 23 2023 10:21AM,258035,10,MedStone Pharma LLC,,,1
6,Feb 23 2023 9:52AM,258031,10,ISDIN Corporation,,,7
7,Feb 23 2023 9:43AM,258030,10,"Methapharm, Inc.",,,3
8,Feb 23 2023 9:41AM,258029,20,Granules Consumer Health,,,1
9,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 11:13AM,258042,10,"Citieffe, Inc.",1,,
1,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation,1,,
2,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.,20,,
3,Feb 23 2023 10:55AM,258038,15,Lonza,1,,
4,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC,5,,
5,Feb 23 2023 10:21AM,258035,10,MedStone Pharma LLC,1,,
6,Feb 23 2023 9:52AM,258031,10,ISDIN Corporation,7,,
7,Feb 23 2023 9:43AM,258030,10,"Methapharm, Inc.",3,,
8,Feb 23 2023 9:41AM,258029,20,Granules Consumer Health,1,,
9,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 11:13AM,258042,10,"Citieffe, Inc.",1,,
1,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation,1,,
2,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.,20,,
3,Feb 23 2023 10:55AM,258038,15,Lonza,1,,
4,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 11:13AM,258042,10,"Citieffe, Inc.",1,NaN,NaN
1,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation,1,NaN,NaN
2,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.,20,NaN,NaN
3,Feb 23 2023 10:55AM,258038,15,Lonza,1,NaN,NaN
4,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC,5,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 11:13AM,258042,10,"Citieffe, Inc.",1,0.0,0.0
1,Feb 23 2023 11:01AM,258041,10,ISDIN Corporation,1,0.0,0.0
2,Feb 23 2023 11:00AM,258039,10,Eywa Pharma Inc.,20,0.0,0.0
3,Feb 23 2023 10:55AM,258038,15,Lonza,1,0.0,0.0
4,Feb 23 2023 10:21AM,258037,10,MedStone Pharma LLC,5,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5160377,164,6.0,3.0
15,2063701,39,29.0,0.0
16,258010,1,0.0,0.0
19,258001,15,21.0,0.0
20,258029,1,0.0,0.0
21,258000,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5160377,164,6.0,3.0
1,15,2063701,39,29.0,0.0
2,16,258010,1,0.0,0.0
3,19,258001,15,21.0,0.0
4,20,258029,1,0.0,0.0
5,21,258000,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,164,6.0,3.0
1,15,39,29.0,0.0
2,16,1,0.0,0.0
3,19,15,21.0,0.0
4,20,1,0.0,0.0
5,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,164.0
1,15,Released,39.0
2,16,Released,1.0
3,19,Released,15.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0
Executing,6.0,29.0,0.0,21.0,0.0,0.0
Released,164.0,39.0,1.0,15.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,29.0,0.0,21.0,0.0,0.0
2,Released,164.0,39.0,1.0,15.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,29.0,0.0,21.0,0.0,0.0
2,Released,164.0,39.0,1.0,15.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()